In [1]:
import pandas as pd
from sklearn.feature_selection import RFE
import numpy as np
from sklearn import model_selection
from xgboost import XGBClassifier as XGBC
import data_read as dr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import os

In [2]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
# 数据存放路径
data_path = base_path + r"\data\01feature\all\12ALOS_BI.csv"
out_path = base_path + r"\result\feature_select\sccheme12.xlsx"

In [3]:
data = dr.read_data_from_csv(data_path)
data

,CLASS_NAME,HH,HV,HH-HV,HH+HV,HH/HV
0,0,-11.258067,-20.599403,9.341336,-15.928736,0.546524
1,0,-12.911981,-20.147547,7.235566,-16.529764,0.640871
2,0,-14.212379,-19.425264,5.212885,-16.818820,0.731644
3,0,-14.782971,-20.602303,5.819331,-17.692636,0.717540
4,0,-12.580205,-20.205948,7.625743,-16.393076,0.622599
...,...,...,...,...,...,...
8660,4,-17.227133,-24.247137,7.020004,-20.737135,0.710481
8661,4,-17.804178,-22.565561,4.761383,-20.184870,0.788998
8662,4,-18.006405,-23.316122,5.309717,-20.661263,0.772273
8663,4,-14.846490,-20.540106,5.693616,-17.693298,0.722805


In [4]:
data_ = data.copy()  # 保护原始数据不收破坏
data_ = data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
x = data_.iloc[:, 1:]
y = data_.iloc[:, 0]
s_num = x.shape[0]  # 返回行数，获取样本数量
f_num = x.shape[1]  # 返回列数，获取特征数量
f_num

5

In [5]:
cv_result = [[] for k in range(20)]
f_support = [[] for k in range(20)]
rank_results = [[] for k in range(20)]
for j in range(0,20):
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33, random_state=j, stratify=y)
    estimator = XGBC(n_estimators=500, eval_metric='mlogloss', random_state=59)
    cv_result_ = []
    f_support_ = []
    rank_results_ = []
    for i in range (0, f_num):
        selector = RFE(estimator, n_features_to_select=i+1, step=1)
        selector = selector.fit(xtrain,ytrain)
        f_support_.append(selector.support_)
        # print(f_support_)
        rank_results_.append(selector.ranking_)
        # print(rank_results_)
        x_warpper = selector.transform(xtrain)
        once = cross_val_score(estimator,x_warpper,ytrain,cv=5).mean()
        cv_result_.append(once)

    cv_result[j].append(cv_result_)
    f_support[j].append(f_support_)
    rank_results[j].append(rank_results_)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [9]:
f_support1=np.array(f_support).reshape([f_num*f_num,20])
cv_result1=np.array(cv_result).reshape([20,f_num])
rank_results1=np.array(rank_results).reshape([f_num*f_num,20])
cv_out=pd.DataFrame(cv_result1)
support_out=pd.DataFrame(f_support1)
rank_out=pd.DataFrame(rank_results1)

In [10]:
with pd.ExcelWriter(out_path, mode='w', engine='openpyxl') as writer:
    cv_out.to_excel(writer, sheet_name='cv', index=True, startcol=0, startrow=0)
    support_out.to_excel(writer, sheet_name='support', index=True, startcol=0, startrow=0)
    rank_out.to_excel(writer, sheet_name='rank', index=True, startcol=0, startrow=0)